# Import Thư Viện

In [576]:
pip install transformers

In [577]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
import multiprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import json
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from gensim.models import Word2Vec
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import accuracy_score

# Load dataset

In [578]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ProjectsAI_CodeInjection/Dataset

data_source = pd.read_csv('data_source.csv')
data_target  = pd.read_csv('data_target.csv')

data_target = data_target.sample(frac=1, random_state=42)

X_source = data_source['text'].str.replace('/',' ')
y_source = data_source['label']
X_target  = data_target ['text'].str.replace('/','')
y_target  = data_target ['label']
rlist =['194 - Fake the Source of Data',
        '66 - SQL Injection',
       '34 - HTTP Response Splitting',
        '126 - Path Traversal',
       '000 - Normal',
        '272 - Protocol Manipulation',
       '310 - Scanning for Vulnerable Software',
        '242 - Code Injection',
       '153 - Input Data Manipulation']
mapping = {l: i+1 for i, l in enumerate(rlist)}
y_source = [mapping[s] for s in y_source]
y_target  = [mapping[r] for r in y_target ]
y_source = np.array(y_source)
y_target  = np.array(y_target )


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/ProjectsAI_CodeInjection/Dataset


### Convert Dữ liệu chưa biết có nhãn là 0, đã biết nhãn là 1

In [579]:
y_target = np.where(np.logical_or(y_target == 1, y_target == 3), 0, 1)

In [580]:
y_target

array([1, 1, 1, ..., 0, 1, 1])

# Feature Extraction

In [581]:
class Vectorizer:
    def __init__(self, method='BOW', ngram_range=(1, 1), max_features=300, emb_fname='', word_index_fname=''):
        self.method = method
        if self.method == 'BOW':
            self.vectorizer = CountVectorizer(analyzer='word', input='content', ngram_range=ngram_range, max_features=max_features)
        elif self.method == 'TFIDF':
            self.vectorizer = TfidfVectorizer(analyzer='word', input='content', max_features=max_features)
        elif self.method == 'Word2Vec':
            self.max_features = max_features
            self.emb_fname = emb_fname
            self.word_index_fname = word_index_fname
        else:
            raise ValueError('Feature extraction method does not exist.')

    def feature_extraction(self, X_train, X_test):
        train_data = self.vectorizer.fit_transform(X_train).toarray()
        test_data = self.vectorizer.transform(X_test).toarray()
        return train_data, test_data

    def get_word_index(self):
        word2id = json.load(open(self.word_index_fname, 'r'))
        return word2id

    def get_embedding_matrix(self):
        np.random.seed(0)
        word2id = self.get_word_index()
        embedding_matrix = np.random.uniform(-0.25, 0.25, [len(word2id) + 1, self.max_features])
        with open(self.emb_fname, 'r', encoding='utf-8') as f:
            for line in f:
                content = line.split(' ')
                if content[0] in word2id:
                    embedding_matrix[word2id[content[0]]] = np.array(list(map(float, content[1:])))
        return embedding_matrix

## BOW

In [582]:
# vectorizer = Vectorizer('BOW')
# X_train, X_test = vectorizer.feature_extraction(X_source, X_target)

## TFTDF

In [583]:
# vectorizer = Vectorizer('TFIDF')
# X_train, X_test = vectorizer.feature_extraction(X_source, X_target)

## WORD2VEC

In [584]:

sentences = [sentence.split() for sentence in X_source]
w2v_model = Word2Vec(sentences, window=5, min_count=5, workers=4)

def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train = np.array([vectorize(sentence) for sentence in X_source])
X_test = np.array([vectorize(sentence) for sentence in X_target])

# Sử dụng GPU

In [585]:
torch.cuda.is_available()

True

In [586]:
device = ""
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# Set các giá trị đầu vào

In [587]:
input_layer = X_train.shape[1]
input_size = input_layer
hidden_size = 64
output_size=  768
num_layers = 2

# Hàm Discriminator

In [588]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_layer, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(x.size(0), input_layer)
        output = self.model(x)
        return output

discriminator = Discriminator().to(device=device)

# Hàm Generator

In [589]:
# class Generator(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.model = nn.Sequential(
#             nn.Linear(input_layer, 256),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(256, 512),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(512, 1024),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(1024, input_layer),
#             nn.Tanh(),
#         )

#     def forward(self, x):
#         output = self.model(x)
#         output = output.view(x.size(0), input_layer)
#         return output

# generator = Generator().to(device=device)



In [590]:
class Generator(nn.Module):

    def __init__(self):
        super(Generator, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)  #, bidirectional = True)
        self.dense_layer1 = nn.Linear(int(hidden_size), 2*int(hidden_size))
        self.dense_layer2 = nn.Linear(2*int(hidden_size), 4*int(hidden_size))
        self.dense_layer3 = nn.Linear(4*int(hidden_size), int(input_size))
        self.dropout = nn.Dropout(0.3)
        self.activation = nn.ReLU()

    def forward(self, x):
        rnn_out, _ = self.lstm(x)
        output = rnn_out
        output = self.dense_layer1(output)
        output = self.activation(output)
        output = self.dropout(output)
        output = self.dense_layer2(output)
        output = self.activation(output)
        output = self.dropout(output)
        output = self.dense_layer3(output)
        output = torch.tanh(output)
        return output

generator = Generator().to(device=device)

# Khởi tạo các giá trị laerning rate, epochs, batch_size, hàm loss, optimization

In [591]:
lr = 0.0001
num_epochs = 4000
batch_size= 32
loss_function = nn.BCELoss()
optimizer_discriminator = torch.optim.Adam(discriminator.parameters(), lr=lr)
optimizer_generator = torch.optim.Adam(generator.parameters(), lr=lr)

# Dùng để chuyển đổi dạng đầu vào nếu dùng trích xuất là Bert

In [592]:
# X_source = X_train.to(device=device, dtype=torch.float32)
# X_target = X_test.to(device=device, dtype=torch.float32)

In [593]:
X_source = torch.from_numpy(X_train).to(device=device, dtype=torch.float32)
X_target = torch.from_numpy(X_test).to(device=device, dtype=torch.float32)

# Tranining

In [594]:
for epoch in range(num_epochs):
    idx = np.random.randint(0, X_source.shape[0], batch_size)
    real_samples  = X_source[idx]
    real_samples_labels = torch.ones((batch_size,1)).to(device=device)
    latent_space_samples = torch.rand((batch_size,input_layer)).to( device=device)

    generated_samples = generator(latent_space_samples)
    generated_samples_labels = torch.zeros((batch_size, 1)).to(
            device=device
        )
    all_samples = torch.cat((real_samples, generated_samples))
    all_samples_labels = torch.cat(
            (real_samples_labels, generated_samples_labels)
        )

        # Training the discriminator
    discriminator.zero_grad()

    output_discriminator = discriminator(all_samples)
    all_samples_labels = all_samples_labels.view(-1, 1)
    loss_discriminator = loss_function(
            output_discriminator, all_samples_labels
        )
    loss_discriminator.backward()
    optimizer_discriminator.step()

        # Data for training the generator
    latent_space_samples = torch.rand((batch_size,input_layer)).to(device=device)

        # Training the generator
    generator.zero_grad()
    generated_samples = generator(latent_space_samples)
    output_discriminator_generated = discriminator(generated_samples)
    loss_generator = loss_function(
            output_discriminator_generated, real_samples_labels
        )
    loss_generator.backward()
    optimizer_generator.step()
    if(epoch%200==0):
        predicted_labels = discriminator(X_target)
        new_tensor = torch.where(predicted_labels >= 0.5, torch.tensor(1), torch.tensor(0))
        new_label = new_tensor.detach().cpu().numpy()

        print(f"accuracy: {accuracy_score(y_target, new_label)}")
        # print(f"Epoch: {epoch/100} Loss D.: {loss_discriminator}")
        # print(f"Epoch: {epoch/100} Loss G.: {loss_generator}")


accuracy: 0.3708124643853283
accuracy: 0.6294574573373721
accuracy: 0.6293974747323277
accuracy: 0.6294574573373721
accuracy: 0.6294274660348499
accuracy: 0.6294574573373721
accuracy: 0.6294574573373721
accuracy: 0.6294574573373721
accuracy: 0.6273880574633356
accuracy: 0.6294574573373721
accuracy: 0.6294574573373721
accuracy: 0.6294274660348499
accuracy: 0.6294574573373721
accuracy: 0.6294574573373721
accuracy: 0.6294574573373721
accuracy: 0.6294574573373721
accuracy: 0.6294874486398945
accuracy: 0.6294574573373721
accuracy: 0.6295174399424167
accuracy: 0.6295174399424167


# chuyển tensor output về 0 và 1

In [595]:
predicted_labels = discriminator(X_target)
new_tensor = torch.where(predicted_labels >= 0.5, torch.tensor(1), torch.tensor(0))

In [596]:
new_label = new_tensor.detach().cpu().numpy()

# Kết quả phân loại trên tập test

In [597]:
accuracy = accuracy_score(y_target, new_label)
accuracy

0.6295174399424167

In [598]:
print('accuracy =', accuracy_score(y_target, new_label))
print(classification_report(y_target,new_label))
print('Confusion Matrix: \n',confusion_matrix(y_target, new_label))

accuracy = 0.6295174399424167
              precision    recall  f1-score   support

           0       0.50      0.00      0.00     12353
           1       0.63      1.00      0.77     20990

    accuracy                           0.63     33343
   macro avg       0.56      0.50      0.39     33343
weighted avg       0.58      0.63      0.49     33343

Confusion Matrix: 
 [[    2 12351]
 [    2 20988]]
